# System info
This code is taken from [here](https://thepythoncode.com/article/get-hardware-system-information-python?utm_content=cmp-true)

With this script I want to get all data from system

## Dependencies
- psutil
- gputil
- tabulate (this one it's only for print gpu data)

In [2]:
import psutil
import platform
from datetime import datetime

def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [3]:
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

======================================== System Information ========================================
System: Linux
Node Name: brian-Sobremesa
Release: 6.2.0-36-generic
Version: #37~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  9 15:34:04 UTC 2
Machine: x86_64
Processor: x86_64


In [4]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2023/11/16 7:24:46


In [5]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 6
Total cores: 12
Max Frequency: 3800.00Mhz
Min Frequency: 2200.00Mhz
Current Frequency: 2992.02Mhz
CPU Usage Per Core:
Core 0: 9.9%
Core 1: 10.2%
Core 2: 3.1%
Core 3: 5.9%
Core 4: 9.0%
Core 5: 4.0%
Core 6: 13.1%
Core 7: 14.9%
Core 8: 9.8%
Core 9: 10.2%
Core 10: 8.0%
Core 11: 14.9%
Total CPU Usage: 6.9%


In [6]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 31.28GB
Available: 25.91GB
Used: 4.72GB
Percentage: 17.2%
==================== SWAP ====================
Total: 0.00B
Free: 0.00B
Used: 0.00B
Percentage: 0.0%


In [7]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: /dev/sdb2 ===
  Mountpoint: /
  File system type: ext4
  Total Size: 915.73GB
  Used: 66.85GB
  Free: 802.29GB
  Percentage: 7.7%
=== Device: /dev/loop0 ===
  Mountpoint: /snap/bare/5
  File system type: squashfs
  Total Size: 128.00KB
  Used: 128.00KB
  Free: 0.00B
  Percentage: 100.0%
=== Device: /dev/loop1 ===
  Mountpoint: /snap/core/16202
  File system type: squashfs
  Total Size: 105.88MB
  Used: 105.88MB
  Free: 0.00B
  Percentage: 100.0%
=== Device: /dev/loop2 ===
  Mountpoint: /snap/core18/2790
  File system type: squashfs
  Total Size: 55.75MB
  Used: 55.75MB
  Free: 0.00B
  Percentage: 100.0%
=== Device: /dev/loop3 ===
  Mountpoint: /snap/core18/2796
  File system type: squashfs
  Total Size: 55.75MB
  Used: 55.75MB
  Free: 0.00B
  Percentage: 100.0%
=== Device: /dev/loop4 ===
  Mountpoint: /snap/core20/1974
  File system type: squashfs
  Total

In [8]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: lo ===
  IP Address: 127.0.0.1
  Netmask: 255.0.0.0
  Broadcast IP: None
=== Interface: lo ===
=== Interface: lo ===
  MAC Address: 00:00:00:00:00:00
  Netmask: None
  Broadcast MAC: None
=== Interface: enp34s0 ===
  IP Address: 192.168.0.133
  Netmask: 255.255.255.0
  Broadcast IP: 192.168.0.255
=== Interface: enp34s0 ===
=== Interface: enp34s0 ===
  MAC Address: 2c:f0:5d:aa:b5:ae
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff:ff:ff
=== Interface: docker0 ===
  IP Address: 172.17.0.1
  Netmask: 255.255.0.0
  Broadcast IP: 172.17.255.255
=== Interface: docker0 ===
  MAC Address: 02:42:34:31:02:3e
  Netmask: None
  Broadcast MAC: ff:ff:ff:ff:ff:ff
Total Bytes Sent: 32.65MB
Total Bytes Received: 249.49MB


In [9]:
# GPU information
import GPUtil
from tabulate import tabulate
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))

======================================== GPU Details ========================================
  id  name                     load    free memory    used memory    total memory    temperature    uuid
----  -----------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  NVIDIA GeForce GTX 1650  33.0%   3198.0MB       697.0MB        4096.0MB        34.0 °C        GPU-9f2ebddb-639b-4d69-b02d-2a70952383c7
